ref: https://www.dataquest.io/blog/pandas-big-data/

In [1]:
import pandas as pd

In [2]:
sample_pickle = "D:\\Igor\\Research_USF\\University of South Florida\\Mao, Wenbin - Igor\\Febio-Models\\Active-Models\\PAQ\\Gamma-5-2\\runs\\0-2\\runs\\pickles\\tpm\\data-run-0.pickle"

In [3]:
df = pd.read_pickle(sample_pickle)

In [4]:
df.head()

,x,y,z,ux,uy,uz,node,timestep,run_ref,param_val,sx,sy,sz,sxy,sxz,syz,elem
0,0.006057,0.002620,-65.4601,0.006057,0.002620,-0.460083,1.0,0.004,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.004150,0.001450,-75.4548,-0.004150,0.001450,-0.454839,2.0,0.004,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34.431600,-0.089488,20.0000,0.698981,-0.089488,0.000000,3.0,0.004,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24.728400,0.199886,20.0000,0.941239,0.199886,0.000000,4.0,0.004,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.003990,0.002142,-67.1256,0.003990,0.002142,-0.458888,5.0,0.004,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933038 entries, 0 to 3933037
Data columns (total 17 columns):
 #   Column     Dtype  
---  ------     -----  
 0   x          float64
 1   y          float64
 2   z          float64
 3   ux         float64
 4   uy         float64
 5   uz         float64
 6   node       float64
 7   timestep   float64
 8   run_ref    int64  
 9   param_val  float64
 10  sx         float64
 11  sy         float64
 12  sz         float64
 13  sxy        float64
 14  sxz        float64
 15  syz        float64
 16  elem       float64
dtypes: float64(16), int64(1)
memory usage: 510.1 MB


In [6]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [7]:
df_int = df.select_dtypes(include=['int'])
converted_int = df_int.apply(pd.to_numeric,downcast='unsigned')
print(mem_usage(df_int))
print(mem_usage(converted_int))
compare_ints = pd.concat([df_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

0.00 MB
0.00 MB


,before,after


In [9]:
df_float = df.select_dtypes(include=['float'])
converted_float = df_float.apply(pd.to_numeric,downcast='float')
print(mem_usage(df_float))
print(mem_usage(converted_float))
compare_floats = pd.concat([df_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

480.11 MB
240.05 MB


,before,after
float32,NaN,16.0
float64,16.0,NaN


In [11]:
optimized_df = df.copy()
optimized_df[converted_int.columns] = converted_int
optimized_df[converted_float.columns] = converted_float
print(mem_usage(df))
print(mem_usage(optimized_df))

510.11 MB
270.06 MB


In [13]:
optimized_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933038 entries, 0 to 3933037
Data columns (total 17 columns):
 #   Column     Dtype  
---  ------     -----  
 0   x          float32
 1   y          float32
 2   z          float32
 3   ux         float32
 4   uy         float32
 5   uz         float32
 6   node       float32
 7   timestep   float32
 8   run_ref    int64  
 9   param_val  float32
 10  sx         float32
 11  sy         float32
 12  sz         float32
 13  sxy        float32
 14  sxz        float32
 15  syz        float32
 16  elem       float32
dtypes: float32(16), int64(1)
memory usage: 270.1 MB


In [17]:
optimized_df.loc[:,"run_ref"] = optimized_df["run_ref"].astype('uint8')

In [18]:
print(mem_usage(df))
print(mem_usage(optimized_df))

510.11 MB
243.80 MB


In [44]:
df_obj = df[["elem", "node", "param_val"]]

converted_obj = pd.DataFrame()
for col in df_obj.columns:
    num_unique_values = len(df_obj[col].unique())
    num_total_values = len(df_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = df_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = df_obj[col]

In [45]:
print(mem_usage(df_obj))
print(mem_usage(converted_obj))
compare_obj = pd.concat([df_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

90.02 MB
30.17 MB


,before,after
float64,3.0,NaN
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0


In [46]:
optimized_df[converted_obj.columns] = converted_obj
print(mem_usage(df))
print(mem_usage(optimized_df))

510.11 MB
228.97 MB


In [42]:
dtypes = optimized_df.dtypes
dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]
column_types = dict(zip(dtypes_col, dtypes_type))


# rather than print all 161 items, we'll
# sample 10 key/value pairs from the dict
# and print it nicely using prettyprint
# preview = first2pairs = {key:value for key,value in list(column_types.items())[:10]}
# import pprintpp
# pp = pp = pprint.PrettyPrinter(indent=4)
# pp.pprint(preview)

In [43]:
column_types

{'x': 'float32',
 'y': 'float32',
 'z': 'float32',
 'ux': 'float32',
 'uy': 'float32',
 'uz': 'float32',
 'node': 'category',
 'timestep': 'float32',
 'run_ref': 'uint8',
 'param_val': 'float32',
 'sx': 'float32',
 'sy': 'float32',
 'sz': 'float32',
 'sxy': 'float32',
 'sxz': 'float32',
 'syz': 'float32',
 'elem': 'category'}

In [47]:
optimized_df

,x,y,z,ux,uy,uz,node,timestep,run_ref,param_val,sx,sy,sz,sxy,sxz,syz,elem
0,0.006057,0.002620,-65.460098,0.006057,0.002620,-0.460083,1.0,0.004,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.004150,0.001450,-75.454803,-0.004150,0.001450,-0.454839,2.0,0.004,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34.431599,-0.089488,20.000000,0.698981,-0.089488,0.000000,3.0,0.004,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24.728399,0.199886,20.000000,0.941239,0.199886,0.000000,4.0,0.004,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.003990,0.002142,-67.125603,0.003990,0.002142,-0.458888,5.0,0.004,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3933033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.190,0,0.0,2.263010,7.46183,9.53921,5.20193,-0.698568,-0.926148,18812.0
3933034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.190,0,0.0,2.009360,9.16172,10.36140,4.14563,-0.323419,-0.569763,18813.0
3933035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.190,0,0.0,0.420362,9.75423,9.58764,3.54162,-0.575045,-1.136300,18814.0
3933036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.190,0,0.0,-0.216978,10.47010,9.35995,2.43152,-0.652445,-1.947120,18815.0


In [48]:
optimized_df["param_val"].describe()

count     3933038.0
unique          1.0
top             0.0
freq      3933038.0
Name: param_val, dtype: float64

In [52]:
optimized_df2 = df.convert_dtypes()
print(mem_usage(df))
print(mem_usage(optimized_df2))

510.11 MB
525.12 MB
